# Preset filters

In this exercise, we will try to improve the model performance by giving the model additional features as input.
Here, we will use rather simple convolutional features and use them as input to the MLP model from the previous
exercise.

## Preparation

In [1]:
# load tensorboard extension
%load_ext tensorboard

In [2]:
# import torch and other libraries
import os
import numpy as np
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from tqdm import trange

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter

In [3]:
!pip install cifar2png

In [4]:
# check if we have gpu support
# colab offers free gpus, however they are not activated by default.
# to activate the gpu, go to 'Runtime->Change runtime type'. 
# Then select 'GPU' in 'Hardware accelerator' and click 'Save'
have_gpu = torch.cuda.is_available()
# we need to define the device for torch, yadda yadda
if have_gpu:
    print("GPU is available")
    device = torch.device('cuda')
else:
    print("GPU is not available, training will run on the CPU")
    device = torch.device('cpu')

GPU is not available, training will run on the CPU


In [ ]:
# run this in google colab to get the utils.py file
!wget https://raw.githubusercontent.com/constantinpape/training-deep-learning-models-for-vison/master/day1/utils.py 

In [5]:
# we will reuse the training function, validation function and
# data preparation from the previous notebook
import utils

In [6]:
cifar_dir = './cifar10'
!cifar2png cifar10 cifar10

output dir `cifar10` already exists. Please specify a different output path


In [7]:
categories = os.listdir('./cifar10/train')
categories.sort()

## Filters

We will now ...

In [ ]:
# apply a list of filters as on the fly transformation. 
def apply_filters(image, target, filter_list, keep_image=False):    
    filtered = [image] if keep_image else [] 
    for filter_function in filter_list:
        filtered.append(filter_function(image))
    data = np.concatenate(filtered, axis=-1)
    return data, target

In [1]:
# this is the initial MLP from the previous exercise
from utils import SimpleMLP

In [ ]:
# build dataset with filter transformations
import skimage.filters as sk_filters

filters = [sk_filters.gaussian, sk_filters.laplace]
filters = partial(apply_filters,
                  filter_list=filters,
                  keep_image=True)

trafos = [
    filters,
    utils.normalize,
    utils.to_channel_first,
    utils.to_tensor
]
trafos = partial(utils.compose, transforms=trafos)

train_dataset = utils.DatasetWithTransform(train_images, train_labels,
                                           transform=trafos)
val_dataset = utils.DatasetWithTransform(val_images, val_labels,
                                         transform=trafos)

In [2]:
# instantiate the model
# note: we have more input features now;
# for each filter 3 additional channels are added to the model
n_input_channels = n_filters * 3 * 32 * 32
model = SimpleMLP(n_input_channels, 10)

NameError: name 'n_filters' is not defined

In [ ]:
# make loaders for training and validation data
batch_size = 8
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size, 
                          shuffle=True,
                          n_workers=8)

val_loader = DataLoader(val_dataset, batch_size=10)

In [ ]:
# run training for the model with preset filters

n_pixels = 9 * 32 * 32  # number channels * number pixels
n_classes = 10    
model = LogisticRegressor(n_pixels, n_classes)
model.to(device)

optimizer = Adam(model.parameters(), lr=1.e-3)

# you can find the results of this training run in the tensorboard
# above as well, they will have the name 'log_reg_filters1' and
# will be differently colored compared to the first model
tb_logger = SummaryWriter('runs/log_reg_filters1')

n_epochs = 4
for epoch in trange(n_epochs):
    train(model, train_loader, loss_function, optimizer,
          device, epoch, tb_logger=tb_logger, log_image_interval=None)
    step = (epoch + 1) * len(train_loader)
    validate(model, val_loader, loss_function, device, step,
             tb_logger=tb_logger)

In [ ]:
# evaluate the new model
test_dataset = utils.DatasetWithTransform(test_images, test_labels,
                                          transform=trafos)
test_loader = DataLoader(test_dataset, batch_size=25)

test_predictions, test_labels = validate(model, test_loader, loss_function,
                                         device, 0, tb_logger=None)
accuracy = metrics.accuracy_score(test_labels, test_predictions)
print("Test accuracy")
print(accuracy)
print()

fig, ax = plt.subplots(1, figsize=(8, 8))
utils.make_confusion_matrix(test_labels,
                            test_predictions,
                            categories, ax)

Advanced:
- The filters we have used here can be expressed as convolutions.
- Express the `gaussian` filter using [nn.Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) and train a model using these filters.
- Can you also explace the `laplace` filter?